In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

### Проверка полученного среза словаря на тесте

### Вспомогательные функции для парсинга

In [2]:
import re

def cleanhtml(raw_html):
    # убрать html-теги
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [3]:
def words(stringIterable):
    #разбивает файл (поток ввода) на слова
    lineStream = iter(stringIterable)
    for line in lineStream: #enumerate the lines
        line = cleanhtml(line)
        for word in line.split(): #further break them down
            if re.match("^[A-Za-z]+[-]?[A-Za-z0-9]+$", word):
                yield word

### Сборка тестового датафрейма

In [4]:
# собирает в массив все слова из позитивных отзывов
import os
a = []
file_list = os.listdir(path='aclImdb/test/pos')
i = 0
for fl in file_list:
    ball = int(fl[fl.find('_')+1 : fl.find('.')])
    f = open('aclImdb/test/pos/' + fl, encoding="UTF-8")
    a += [[w.strip('!?,.:;()"').lower(), 1, 0, fl, ball] for w in words(f)]
    f.close()
    i +=1
    if i%3000 == 0:
        # индикатор процесса
        print(i)
    

3000
6000
9000
12000


In [5]:
# добирает в массив все слова из негативных отзывов
file_list = os.listdir(path='aclImdb/test/neg')
i = 0
for fl in file_list:
    ball = int(fl[fl.find('_')+1 : fl.find('.')])
    f = open('aclImdb/test/neg/' + fl, encoding="UTF-8")
    a += [[w.strip('!?,.:;()"').lower(), 0, 1, fl, ball] for w in words(f)]
    f.close()
    i +=1
    if i%3000 == 0:
        print(i)

3000
6000
9000
12000


In [6]:
df = pd.DataFrame(a, columns =['word', 'ispos', 'isneg', 'comm_file', 'score'])

In [7]:
df.shape

(4532530, 5)

In [8]:
df[['ispos', 'isneg']].sum()

ispos    2279567
isneg    2252963
dtype: int64

### Формирование словаря частотностей

In [9]:
v_slice = pd.read_csv('v_slice.csv')

### Вычисление веса отзывов

In [10]:
res = df.merge(v_slice, on=['word'], how='inner')

In [11]:
res.shape # для построения метрики используется менее 10% слов базового датафрейма:

(411547, 10)

In [12]:
res1 = res.groupby('comm_file').agg({'pos': [np.mean], 'score': [np.max]}).reset_index()
res1.shape

(24942, 3)

### Расчет итоговой корреляции

In [13]:
res1.corr()

,,pos,score
,,mean,amax
pos,mean,1.000000,0.651868
score,amax,0.651868,1.000000
